In [32]:
#import data

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


In [33]:
#read csv
df = pd.read_csv('creditcard_cc_fruad.csv')
df.head()   #seee first few column

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [34]:
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.") #see how many data in dataset

The dataset has 284807 rows and 31 columns.


In [35]:
#split feature and Class

x = df.drop('Class', axis=1)
y = df['Class']

shape = x.shape 
num_of_features = shape[1]
print(num_of_features) #see how many x feature in the dataset

30


In [36]:
# Split into training and testing datasets by train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123, stratify=y)

In [37]:
#convert dataframe to tensor
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).long() #to long as using CrossEntropyLoss, label need to be long
x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).long()

#tensor to tensor dataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

#tensor dataset to dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [38]:
#decoder for classify two class from features output two logic
#30 features ->64 ->relu ->48 ->relu ->32 ->relu ->2 , in interface, argmax at last

class CcFruadClassifier(nn.Module):
    def __init__(self, num_of_features):
        super(CcFruadClassifier, self).__init__()
        self.fc1 = nn.Linear(num_of_features, 64)
        self.fc2 = nn.Linear(64, 48)
        self.fc3 = nn.Linear(48, 32)
        self.fc4 = nn.Linear(32, 2)  # Output layer for two-class classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [39]:
# check if can use cuda, no cud use cpu instead
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CcFruadClassifier(num_of_features).to(device) # to gpu orcpu
loss_formula = torch.nn.CrossEntropyLoss() #use cross entropy loss for binary classification can also be use for multi-class
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [40]:
epochs = 200

#start training model
for epoch in range(epochs):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_formula(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_dataloader):.4f}")

    # Evaluate on test set
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += loss_formula(outputs, labels).item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Test Loss: {test_loss/len(test_dataloader):.4f}")

    if epoch %20 == 0:
        torch.save(model.state_dict(), "checkpoint{}.pt".format(epoch+1))
torch.save(model.state_dict(), "checkpoint{}.pt".format(epoch+1))   #save last epoch result

Epoch [1/200], Train Loss: 1.7480
Epoch [1/200], Test Loss: 0.0653
Epoch [2/200], Train Loss: 0.0967
Epoch [2/200], Test Loss: 0.0413
Epoch [3/200], Train Loss: 0.0834
Epoch [3/200], Test Loss: 0.2721
Epoch [4/200], Train Loss: 0.0604
Epoch [4/200], Test Loss: 0.5715
Epoch [5/200], Train Loss: 0.1139
Epoch [5/200], Test Loss: 0.3613
Epoch [6/200], Train Loss: 0.0650
Epoch [6/200], Test Loss: 0.1691
Epoch [7/200], Train Loss: 0.0422
Epoch [7/200], Test Loss: 0.1159
Epoch [8/200], Train Loss: 0.0313
Epoch [8/200], Test Loss: 0.0299
Epoch [9/200], Train Loss: 0.0416
Epoch [9/200], Test Loss: 0.0168
Epoch [10/200], Train Loss: 0.0283
Epoch [10/200], Test Loss: 0.0199
Epoch [11/200], Train Loss: 0.0210
Epoch [11/200], Test Loss: 0.0154
Epoch [12/200], Train Loss: 0.0176
Epoch [12/200], Test Loss: 0.0126
Epoch [13/200], Train Loss: 0.0185
Epoch [13/200], Test Loss: 0.0128
Epoch [14/200], Train Loss: 0.0138
Epoch [14/200], Test Loss: 0.0144
Epoch [15/200], Train Loss: 0.0127
Epoch [15/200], T